<h1 style = "font-size: 35px; text-align: center;">AI CA2</h1>
<h1 style = "font-size: 35px; text-align: center;">Logical Gate Analysis with Genetic Algorithm</h1>
<h1 style = "font-size: 32px; text-align: center; color: #666">Hoomch 810196443</h1>

<br>
<h1 style = "text-align: center">Prelude</h1>
<br>
<h2>Definition of project:</h2>
<p style = "font-size: 14px">All we have as input is a csv file containing a look-up table which represents a series of gates. It is worth mentioning that we are using genetic methods here since normal methods such as testing are actually impossible due to the magtitude of the problem. (There are over 60 million possible candidates) <br> </p>

## Importing Needed Libraries:

In [1]:
import pandas as pd
import numpy as np
from numpy.random import choice
import random

### 1: Reading from input file:
<p>As briefly discussed above, there's only one input file named <mark> truth_table.csv </mark></p>
<p>Please note that all True/False values are replaced with their corresponding <mark> int </mark> values in order to preclude furthur need for conversion in other methods </p>

In [2]:
table = pd.read_csv('truth_table1.csv')
table = table.astype(int)
table_list = table.values.tolist()
table.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Output
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,1,1,1
4,0,0,0,0,0,0,1,0,0,1


### 2: Using dictionary for gates
<p> Cell below is a simple mapping from gate names to their index. From this point forward, each gate will be recognized only with it's code </p>

In [3]:
reference = {1: 'and', 2: 'or', 3: 'xor',
            4:'nand', 5:'nor', 6:'xnor'} 
reference_list = [1,2,3,4,5,6]

### 3: Implementing logic of each gate:
<p> Method below uses built in python operators to implement logic for each one of the gates present in project. </p>
<div style = "align-items : center; margin-top: 10px">
<img src = "img1.png">
</div>

In [4]:
def calc_output(a, b, element_type):
    if (element_type == 'and'):
        return (a & b)
    elif (element_type == 'or'):
        return (a | b)
    elif (element_type == 'xor'):
        return (a ^ b)
    elif (element_type == 'nand'):
        return int((not (a & b)))
    elif (element_type == 'nor'):
        return int((not (a | b)))
    elif (element_type == 'xnor'):
        return int((not (a ^ b)))

### 4: Calculate potency of each answer
<p> Method below is responsible for returning the fitness of each answer.Through this project, Each possible answer will be compared to each one of the rows present in truth table; result of each comparison is either one or zero. Sum of all the points gathered for all rows will determine relative fitness of that gate list. Each answer of which it's corresponding result is equal to the number of rows in truth table can be considered as a final output.</p>   

In [5]:
def calc_point(gate_list):
    size = len(table_list[0])
    count = 0
    for row in table_list:
        out1 = calc_output(row[0], row[1], reference[gate_list[0]])
        temp_out = out1
        new_out = 0
        for i in range(1,8):
            new_out = calc_output(row[i+1], temp_out, reference[gate_list[i]])
            temp_out = new_out
        if (new_out == row[size - 1]):
            count += 1
    return count

new_gate = [4, 6, 5, 3, 1, 2, 4, 6, 5]
new_gate2 = [1, 3, 5, 3, 1, 2, 1, 3, 5]
new_gate3 = [4, 6, 2, 6, 1, 2, 4, 6, 5]
gate_list = [1,3,2,6,1,2,1,3,5]
calc_point(new_gate3)    

128

<br>
<h1 style= "text-align: center">Process steps </h1>
<h2>Definition of chromosomes:</h2>
<p>Each chromosome is a list which contains N elements. N is the number of gates needed to assemble the circuit. For the purpose of this example it is set to 9. It is obvious that Each gene is a single digit that represent a gate. Gate name of each gene can be verified by taking a look at <mark>reference dictionary.</mark></p>
<p> For example, chromosome <mark> [4, 6, 5, 3, 1, 2, 4, 6, 5] </mark> is the represenation for this sequence of gates:</p>
<h3 style= "text-align: center"> Nand , Xnor, Nor, Xor, And, Or, Nand, Xnor, Nor </h3>

<h2>Generating initial population:</h2>
<p>numpy random is used to generate initial population. Population size for all generations is equal to 500.</p>
<p>In this function in order to have random initial chromosomes, we used <b>np.random</b> library and <b>choice()</b> to create an uniformed list of numbers 0 to 5. Then we added one to all 6 generated numbers to make a valid gate list.</p>

<h2>Fitness function:</h2>
<p><mark>assign_points()</mark> uses <mark>calc_point()</mark> to calculates fitness for required chromosomes:</p>
<p style="text-indent :2em;"> 1. Calculated fitness for chromosme number "i".</p>
<p style="text-indent :2em;"> 2. If the output number is maximum so far, set maximum to this number.</p>
<p style="text-indent :2em;"><b> 3. Continue until input list has been completed. Return a list of points along with max value and it's corresponding index (This way we can save time in finding the final answer)</b></p>
<br>

<h2>Crossover:</h2>
<p><mark>make_cross()</mark> generates 2 new children from 2 parents.</p>
<p> During each iteration of the main loop, 2 parents will be selected using <mark> numpy.choice </mark> method. This function needs a target list (which is basically pop_list) along with a same-size list of probabilities for each list element. These Probs are saves inside the <mark> prob_list </mark>. It is calculated by dividing each element of <mark> point list (list returned by the assign point method) </mark> to the sum of point list.
<p> So after applying the choice method we have got two different indexes that we can use to select our two parents for the current iteration.Last part of the crossover feature is the <mark> make_cross </mark> method. It will get the two parents as input and divide them both into two parts. First part of the first parent along with the second part of the second parent will form the first child. Second child is formed by combining first part of the second parent with the second part of the first parent.</p>
<p> Please note that I have not used a probability for cross over and all chosen parents will definately produce children. To make up for this part I've added the most promising candidates directy to the new_population. </p>


<h2>Mutation:</h2>
<p><mark>mutate()</mark> uses <mark>mutate_prob_list</mark> in order to swap two characters in chromosome:</p>
<p style="text-indent :2em;"> 1. generate a random number between 0 and 1 and check if it is less than <b>mu_p = 0.25</b>.</p>
<p style="text-indent :2em;"> 2. if True: choose 2 points in chromosome and swap them and return new chromosome.</p>
<p style="text-indent :2em;"> 3. if False: don't make any changes in the chromosome.</p>
<p>
<br>
For mutation we had two solutions: <b>1. using one swap as described above. 2. using multiple swap (for example 1 per every character)</b>. First solution is much more faster based on experiments we've done, but local max problem is possible, so we've set a variable called <mark>delta = 120</mark> to check if during 120 seconds, No full answer has been reached, it will add random chromosomes to the new_pop list. and it is still much more faster than solution 2. (in average about 6 times faster)</p>
<br>


<h1 style= "text-align: center">Issues</h1>
<br>

<h2>What if we increase population size during iterations?</h2>
<p>Increase of the population size, above 500 chromosomes, does not improve the solution accuracy. Moreover, the computational time is increased significantly.</p>
<br>

<h2>What Is the effect of mutation? Would it be any problem if we use only crossover?</h2>
<p>The most important effect of mutation is to generate new chromosomes using random abilities and to prevent getting stuck in local maximums or at least lower probability of it. So without mutation, we simply don't reach the goal because there would be a kind of bias in chromosomes after tens or hundreds iterations.</p>
<br>

<h2>Mutation or crossover, which one is more effective? Which one increases accuracy faster?</h2>
<p>As we saw experimentally and as it's resonable, <b>mutation</b> is more effective because of the ability of generating new chromosomes even when chromosomes are similar.</p>
<br>

<h2>It's still possible to get stuch in local maxes though we use crossover and mutation. Why?</h2>
<p>If the bias is too string, and it usually happens because of new generation process, mutation and crossover can not afford to change it. So here are some of the solutions:</p>
    <p style="text-indent :2em;"> 1. Increase probability Pc. (Not so good!)</p>
    <p style="text-indent :2em;"> 2. Extention of mutation to all characters.(Too much time overhead)</p>
    <p style="text-indent :2em;"> 3. Use a simple algorithm with low complexity of time and use random chromosomes it when it get stuch in local max.</p>
<br>

In [10]:
import time
def game(pop_size, mu_p, chromo_size, row_size):
    def make_cross(list_1, list_2):
        new_list = []
        new_list2 = []        
        first_iter = int(chromo_size / 2)
        second_iter = first_iter
        
        for i in range(first_iter):
            new_list.append(list_1[i])
            new_list2.append(list_2[i])
        for j in range(second_iter):
            new_list.append(list_2[j+4])
            new_list2.append(list_1[j+4])
        return (new_list, new_list2)
        



    def mutate(crossed_over, i, j):
        new_list = []
        new_list = crossed_over
        new_list[i], new_list[j] = new_list[j], new_list[i]
        return new_list
        


    def assign_points(pop_list):
        point_list = []
        max_score = 0
        index = -1
        for i in range(len(pop_list)):
            score = calc_point(pop_list[i])
            point_list.append(score)
            if (max_score < score):
                max_score = score
                index = i
        return (np.array(point_list), max_score, index)



    pop_list = []
    pop_list2 = []
    pop_hash = set()
    point_list = []
    prob_list = []
    flag = False
    for i in range(pop_size):
        temp = np.random.choice(6, chromo_size)
        new_chromo = [(x+1) for x in temp]
        pop_list.append(new_chromo)
        point = calc_point(new_chromo)
        if (point == row_size):
            flag = True
            return (new_chromo, 20.045)
    
    
    mutate_prob_list = []
    for j in range(100000):
        mutate_prob_list.append(random.sample(range(0, chromo_size), 2))
    
    
    pop_list_sorted = pop_list2
    start = time.time()
    j = 0
    super_max = 0
    for_steps = int(pop_size * 2 / 5)
    strong_steps = int(pop_size / 5)
    use_strongs = False
    got_attention = False
    while(not flag):
        delta = time.time() - start
        if (delta > 240):
            return "Time exceeded!"
        
        new_pop = []
        if (delta > 120 and (got_attention == False)):
            for_steps = for_steps - int (pop_size / 20)
            for m in range(int (pop_size / 10)):
                temp = np.random.choice(6, chromo_size)
                new_chromo = [(x+1) for x in temp]
                new_pop.append(new_chromo)
            got_attention = True
        
        
        point_list, max_score, index = assign_points(pop_list)
        if (max_score == row_size):
            return (pop_list[index], time.time() - start)
            break
        if (super_max < max_score):
            super_max = max_score
        
        plist_sum = np.sum(point_list)
        prob_list = point_list / plist_sum
        pop_list_sorted = [x for _,x in sorted(zip(prob_list,pop_list))]
        

        for i in range(for_steps):
            draw = choice(pop_size, 2, p=prob_list)
            chosen_1 = pop_list[draw[0]]
            chosen_2 = pop_list[draw[1]]
            cross_rand = random.uniform(0, 1)
            crossed_over1, crossed_over2 = make_cross(chosen_1, chosen_2)
            mutation_rand1 = random.uniform(0, 1)
            mutation_rand2 = random.uniform(0, 1)
            probs = mutate_prob_list[j]
            if (mutation_rand1 < mu_p):
                new_pop.append(mutate(crossed_over1, probs[0], probs[1]))
            else:
                new_pop.append(crossed_over1)
            if (mutation_rand2 < mu_p):
                new_pop.append(mutate(crossed_over2, probs[1], probs[0]))
            else:
                new_pop.append(crossed_over2)
        

        for m in range(strong_steps):
            new_pop.append(pop_list_sorted[pop_size - m - 1])

        if (got_attention):
            for_steps = int(pop_size * 2 / 5)
        
        pop_list = new_pop
        last_super_max = super_max
        print(j, super_max, delta)
        j += 1

In [11]:
res = game(200, 0.25, 8, pow(2,9))
print(res)

0 448 0.0
1 458 0.5417032241821289
2 464 1.052370548248291
3 464 1.593677282333374
4 466 2.1046080589294434
5 476 2.616666555404663
6 476 3.1719439029693604
7 476 3.714496374130249
8 476 4.235077619552612
9 476 4.774626970291138
10 476 5.285492658615112
11 476 5.821021556854248
12 478 6.354496240615845
13 478 6.873751878738403
14 478 7.434303283691406
15 478 7.993932247161865
16 478 8.544421195983887
17 478 9.118922710418701
18 478 9.70033049583435
19 478 10.269845008850098
20 478 10.841172456741333
21 478 11.407009363174438
22 482 11.992170333862305
23 482 12.571584939956665
24 492 13.167598485946655
25 496 13.771710634231567
26 496 14.374367475509644
27 496 14.987988471984863
28 496 15.567569017410278
29 496 16.15489149093628
30 496 16.765543937683105
31 508 17.344996690750122
32 508 17.91746211051941
33 508 18.47099256515503
34 508 19.041151523590088
35 508 19.594565868377686
36 508 20.16004991531372
37 508 20.71457290649414
38 508 21.266054391860962
39 508 21.83863663673401
40 508 

## Last Notes:

### Why this fitness?
<p> Since it is the most obvious one to consider and also requires little time to compute.(About 0.003 seconds for each call). Besides a high score using this algorithm means a nearer answer to the final one </p>

### How to choose parents?
<p> I used numpy choice library to choose two parents at the beginning of each iteration. This method will get a source list which is population_list, a count which is 2 and finally a list of probabilities that should be the same size as source list. This method is both fast (since it uses numpy) and also all elements can participate in being a parent which is much better that always choosing the top ranking elements.<p/>